In [1]:
import re
import pandas as pd
import itertools
import cobra.flux_analysis
import time
from functools import partial
from src.assorted_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Recon3D/JSON/Recon3D.json'
model = cobra.io.load_json_model(model_file_path)

SNP_results = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_effect_mod.tsv', index_col=0)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Model load time: 7.488683 seconds


In [21]:
# Prepare SNP combinations, reaction constraints
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations.tsv', index_col=0)
combinations['combinations'] = combinations['combinations'].apply(lambda x: x.split(';'))

combinations['gene_model_ids'] = combinations['combinations']\
    .apply(lambda x: SNP_results.loc[SNP_results['variant_name'].isin(x), ['gene_number']].iloc[:, 0].tolist())

id_list = ';' + ';'.join(model.genes.list_attr('id')) + ';'

combinations['gene_model_ids'] = combinations['gene_model_ids'].apply(lambda x:
                                list(set(itertools.chain.from_iterable(
                                [re.findall(r"(?:;)(" + str(i) + r"_AT\d)", id_list) for i in x]))))

combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

In [22]:
start_time = time.time()


combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)
combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')


end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 135.157937 seconds


In [23]:
# Check for non nominal results
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) != 755:
        print(result)



combinations      [rs11836367, rs115333910, rs6714735, rs5501478...
gene_model_ids    [374875_AT1, 123745_AT1, 5295_AT2, 2954_AT2, 5...
results                         <Solution 746.737 at 0x26fb70ba220>
Name: 24, dtype: object
combinations      [rs6714735, rs532955154, rs149994638, rs143596...
gene_model_ids    [10919_AT1, 5331_AT1, 8704_AT1, 2643_AT1, 8802...
results                         <Solution 650.384 at 0x26fb22cf7c0>
Name: 39, dtype: object
combinations      [rs117583029, rs187679012, rs6714735, rs533568...
gene_model_ids    [23396_AT1, 10840_AT1, 2548_AT1, 54872_AT1, 81...
results                         <Solution 746.737 at 0x26fb6faef40>
Name: 45, dtype: object
combinations      [rs138492057, rs145424036, rs541428129, rs1825...
gene_model_ids    [6507_AT1, 6530_AT1, 23067_AT1, 19_AT1, 50484_...
results                         <Solution 746.737 at 0x26fb27b0850>
Name: 72, dtype: object
combinations      [rs371015963, rs753589371, rs188832143, rs9628...
gene_model_ids    [9

In [ ]:
# Test stuff

def get_model_gene_info():
    ids = model.genes.list_attr('id')
    gene_names = []

    for i in ids:
        gene_names.append(model.genes.get_by_id(i).__getattribute__('name'))

    pd.Series(gene_names).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/gene_names_in_model.tsv', sep='\t')


In [4]:
# Test stuff

#print(model.reactions.HMR_9505.gene_name_reaction_rule)
#model.reactions.MAOX.genes


gene = model.genes.get_by_id('340811_AT1')
print(gene.__getattribute__('name'))
gene

Gene identifier,340811_AT1
Name,
Memory address,0x026fa28a9c70
Functional,True
In 1 reaction(s),HMR_9804


In [20]:
new_SNPs = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/SNPs_all_chrom_orig.tsv')
columns = ['rsids', 'Variant source', 'chrom', 'pos', 'ref', 'alt']
new_SNPs = new_SNPs.reindex(columns=columns)
new_SNPs.rename(columns={'rsids': 'Variant name', 'chrom': 'Chromosome/scaffold name', 'pos': 'Chromosome/scaffold position start (bp)'}, inplace=True)
new_SNPs['Chromosome/scaffold position end (bp)'] = new_SNPs['Chromosome/scaffold position start (bp)']
new_SNPs['Variant alleles'] = new_SNPs['ref'] + '/' + new_SNPs['alt']
new_SNPs.drop(columns=['ref', 'alt'], inplace=True)
new_SNPs['Strand'] = 1
new_SNPs['Chromosome/scaffold position start (bp)'] = new_SNPs['Chromosome/scaffold position start (bp)'].apply(int)
new_SNPs['Chromosome/scaffold position end (bp)'] = new_SNPs['Chromosome/scaffold position end (bp)'].apply(int)
new_SNPs.set_index(['Variant name'], drop=True, inplace=True)
print(new_SNPs.head())

new_SNPs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/SNPs_all_chrom.tsv', sep='\t')

              Variant source  Chromosome/scaffold name  \
Variant name                                             
rs1000036                NaN                        14   
rs1000036                NaN                        14   
rs10000745               NaN                         4   
rs1000108                NaN                         8   
rs10001104               NaN                         4   

              Chromosome/scaffold position start (bp)  \
Variant name                                            
rs1000036                                    22772136   
rs1000036                                    22772136   
rs10000745                                  126429447   
rs1000108                                    27429649   
rs10001104                                    1527546   

              Chromosome/scaffold position end (bp) Variant alleles  Strand  
Variant name                                                                 
rs1000036                            